<a href="https://colab.research.google.com/github/Ahmed1010-1999/deep_learnig/blob/main/neural_network_multi_classification_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [126]:

import torch
from torch import nn
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split
def softmax(input_w: torch.Tensor)->torch.Tensor:
  z=torch.exp(input_w)

  d=torch.sum(z,dim=1)
  d = z / d[:, None]
  return d

# Set the hyperparameters for data creation
NUM_CLASSES = 4
NUM_FEATURES = 2
RANDOM_SEED = 42

# 1. Create multi-class data
X_blob, y_blob = make_blobs(n_samples=1000,
    n_features=NUM_FEATURES, # X features
    centers=NUM_CLASSES, # y labels
    cluster_std=1.5, # give the clusters a little shake up (try changing this to 1.0, the default)
    random_state=RANDOM_SEED
)

# 2. Turn data into tensors
X_blob = torch.from_numpy(X_blob).type(torch.float)
y_blob = torch.from_numpy(y_blob).type(torch.LongTensor)
#print(X_blob[:5], y_blob[:5])

# 3. Split into train and test sets
X_blob_train, X_blob_test, y_blob_train, y_blob_test = train_test_split(X_blob,
    y_blob,
    test_size=0.2,
    random_state=RANDOM_SEED
)
print(y_blob[:30])

tensor([3, 2, 2, 1, 1, 2, 1, 2, 2, 1, 1, 3, 0, 2, 2, 2, 0, 0, 0, 1, 1, 3, 3, 3,
        1, 1, 0, 0, 2, 1])


In [127]:


# Build model
class BlobModel(nn.Module):
    def __init__(self, input_features, output_features, hidden_units=8):
        super().__init__()
        self.linear_layer_stack = nn.Sequential(
            nn.Linear(in_features=input_features, out_features=hidden_units),

            nn.Linear(in_features=hidden_units, out_features=hidden_units),

            nn.Linear(in_features=hidden_units, out_features=output_features),
        )

    def forward(self, x):
        return self.linear_layer_stack(x)

# Create an instance of BlobModel and send it to the target device
model_4 = BlobModel(input_features=NUM_FEATURES,
                    output_features=NUM_CLASSES,
                    hidden_units=8)

In [128]:
# Create loss and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model_4.parameters(),
                            lr=0.1) # exercise: try changing the learning rate here and seeing what happens to the model's performance

In [129]:
# Perform a single forward pass on the data (we'll need to put it to the target device for it to work)
model_4(X_blob_train)[:5]

tensor([[ 1.1010,  0.5867, -0.8381,  0.7735],
        [ 0.9743, -3.1038,  0.5808,  2.0448],
        [-1.7637, -0.7465,  2.3091, -1.8137],
        [-0.9728, -1.0193,  1.6660, -0.8549],
        [ 0.6359, -2.8051,  0.7918,  1.5651]], grad_fn=<SliceBackward0>)

In [130]:
# Make prediction logits with model
y_logits = model_4(X_blob_test)

d=softmax(y_logits[:5])

print(d)
print('===============')
print('===============')
print('===============')
print('===============')
# Perform softmax calculation on logits across dimension 1 to get
y_pred_probs = torch.softmax(y_logits, dim=1)
print(y_logits[:5])
print(y_pred_probs[:5])




tensor([[0.4256, 0.1594, 0.0584, 0.3566],
        [0.0965, 0.0014, 0.5140, 0.3881],
        [0.0342, 0.0981, 0.8390, 0.0287],
        [0.3425, 0.3224, 0.1019, 0.2333],
        [0.2714, 0.0060, 0.0765, 0.6461]], grad_fn=<DivBackward0>)
tensor([[ 1.1940,  0.2118, -0.7930,  1.0172],
        [ 0.0936, -4.1314,  1.7660,  1.4851],
        [-1.3870, -0.3346,  1.8118, -1.5638],
        [ 0.7131,  0.6526, -0.4988,  0.3292],
        [ 1.3155, -2.4986,  0.0485,  2.1827]], grad_fn=<SliceBackward0>)
tensor([[0.4256, 0.1594, 0.0584, 0.3566],
        [0.0965, 0.0014, 0.5140, 0.3881],
        [0.0342, 0.0981, 0.8390, 0.0287],
        [0.3425, 0.3224, 0.1019, 0.2333],
        [0.2714, 0.0060, 0.0765, 0.6461]], grad_fn=<SliceBackward0>)


In [131]:
# Sum the first sample output of the softmax activation function
torch.sum(y_pred_probs[0]),torch.sum(d[0])

(tensor(1., grad_fn=<SumBackward0>), tensor(1., grad_fn=<SumBackward0>))

In [133]:
# Which class does the model think is *most* likely at the index 0 sample?
print(y_pred_probs[0])
print(torch.argmax(y_pred_probs[0]))
print(d[0])
print(torch.argmax(d[0]))


tensor([0.4256, 0.1594, 0.0584, 0.3566], grad_fn=<SelectBackward0>)
tensor(0)
tensor([0.4256, 0.1594, 0.0584, 0.3566], grad_fn=<SelectBackward0>)
tensor(0)


In [134]:
def accuracy_fn(y_true,y_pred):
  correct=torch.eq(y_true,y_pred).sum().item()
  acc=(correct/len(y_pred))*100
  return acc


In [136]:
# Fit the model
torch.manual_seed(42)

# Set number of epochs
epochs = 100



for epoch in range(epochs):
    ### Training
    model_4.train()

    # 1. Forward pass
    y_logits = model_4(X_blob_train) # model outputs raw logits
    y_pred = torch.softmax(y_logits, dim=1).argmax(dim=1) # go from logits -> prediction probabilities -> prediction labels
    # print(y_logits)
    # 2. Calculate loss and accuracy
    loss = loss_fn(y_logits, y_blob_train)
    acc = accuracy_fn(y_true=y_blob_train,
                      y_pred=y_pred)

    # 3. Optimizer zero grad
    optimizer.zero_grad()

    # 4. Loss backwards
    loss.backward()

    # 5. Optimizer step
    optimizer.step()

    ### Testing
    model_4.eval()
    with torch.inference_mode():
      # 1. Forward pass
      test_logits = model_4(X_blob_test)
      test_pred = torch.softmax(test_logits, dim=1).argmax(dim=1)
      # 2. Calculate test loss and accuracy
      test_loss = loss_fn(test_logits, y_blob_test)
      test_acc = accuracy_fn(y_true=y_blob_test,
                             y_pred=test_pred)

    # Print out what's happening
    if epoch % 10 == 0:
        print(f"Epoch: {epoch} | Loss: {loss:.5f}, Acc: {acc:.2f}% | Test Loss: {test_loss:.5f}, Test Acc: {test_acc:.2f}%")

Epoch: 0 | Loss: 0.03255, Acc: 99.12% | Test Loss: 0.02377, Test Acc: 99.50%
Epoch: 10 | Loss: 0.03171, Acc: 99.12% | Test Loss: 0.02279, Test Acc: 99.50%
Epoch: 20 | Loss: 0.03103, Acc: 99.12% | Test Loss: 0.02197, Test Acc: 99.50%
Epoch: 30 | Loss: 0.03046, Acc: 99.12% | Test Loss: 0.02128, Test Acc: 99.50%
Epoch: 40 | Loss: 0.02998, Acc: 99.12% | Test Loss: 0.02068, Test Acc: 99.50%
Epoch: 50 | Loss: 0.02956, Acc: 99.12% | Test Loss: 0.02016, Test Acc: 99.50%
Epoch: 60 | Loss: 0.02920, Acc: 99.12% | Test Loss: 0.01970, Test Acc: 99.50%
Epoch: 70 | Loss: 0.02889, Acc: 99.12% | Test Loss: 0.01929, Test Acc: 99.50%
Epoch: 80 | Loss: 0.02861, Acc: 99.12% | Test Loss: 0.01893, Test Acc: 99.50%
Epoch: 90 | Loss: 0.02836, Acc: 99.12% | Test Loss: 0.01860, Test Acc: 99.50%


In [137]:
def accuracy_fn(y_true,y_pred):
  correct=torch.eq(y_true,y_pred).sum().item()
  acc=(correct/len(y_pred))*100
  return acc


In [138]:
# Turn predicted logits in prediction probabilities
y_pred_probs = torch.softmax(y_logits, dim=1)

# Turn prediction probabilities into prediction labels
y_preds = y_pred_probs.argmax(dim=1)

# Compare first 10 model preds and test labels
print(f"Predictions: {y_preds[:10]}\nLabels: {y_blob_test[:10]}")


Predictions: tensor([1, 0, 2, 2, 0, 0, 0, 1, 3, 0])
Labels: tensor([1, 3, 2, 1, 0, 3, 2, 0, 2, 0])
